# Cloze Filling-基于BiLSTM的完形填空问题-2处预测

In [1]:
print("""-----董政宇 2023年4月23日21:58:58-----""")

-----董政宇 2023年4月23日21:58:58-----


In [2]:
'''
step1: 数据预处理
'''
import os
import torch 
import torch.nn as nn
import torch.utils.data as Data
import random
import torch.utils.data as Data

# 超参数设置
BATCH_SIZE = 32
LR = 1e-4
NUM_HIDDENS = 256
SEQ_LEN = 40
WORD_DIM = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = '../dataTry/biLSTM_data'
# chinese_data_path = os.path.join(path, 'chinese-data.txt')
chinese_data_path = os.path.join(path, 'train.csv')
origin_data,predict_word = [],[]
# vocab
vocab = []
# str to index
s2i = dict()
# index to str
i2s = dict()
index = 0

def get_vocab(path):
    chinese_data_path = path
    with open(chinese_data_path,'r', encoding='utf-8') as f:
        # print('loading txt...')
        for line in f:
            if len(line) >= 41:
            	# 需要对？进行特殊处理
                # print(line)
                line = line.replace("?", "。")
                line = line.replace("？", "。")
                predict_word.append(line[20])
                d = line[:20]+'?'+line[21:40]
                origin_data.append(d)
                for word in line:
                    if word not in vocab:
                        vocab.append(word)
    f.close()
get_vocab(chinese_data_path)

# vocab size
vocab.append('?')
vocab_size = len(vocab)

random.shuffle(vocab)
s2i = {
            word: i for i, word in enumerate(vocab)}
i2s = {
            i :word for i, word in enumerate(vocab)}

In [3]:
vocab_size, s2i['?'], i2s[264], len(predict_word), origin_data[11]

(4241, 2040, '藉', 8458, '1,"入住的是度假区的豪华海景房,前台给?5楼(最高6楼),然后差不多100%的')

In [4]:
origin_data[8450:8458], predict_word[8450:8458]

(['1,《菲菲生气了》告诉孩子，生气是一种很?常的情绪反应，常常是由于自己感到受到了',
  '0,标准内存太小，自己加了一个！没有自带?恢复光盘，重装系统后驱动有问题，自带的',
  '1,"不错,周围很热闹,出行方便,更不愁?吃饭的地方。去浅水湾、赤柱、海洋公园(',
  '1,看过该书，感觉中医暂时不会消亡，尚有?、二十株老树活着，还有毛以林、黄煌、刘',
  '0,这本书没读到底，不是特别喜欢。完全可?用序中的评价来表达我的感受：可以包容，',
  '1,"虽是观景房,不过我住的楼层太低(1?楼)看不到江景,但地点很好,离轻轨临江',
  '1,性价比不错，交通方便。行政楼层感觉很?，只是早上8点楼上装修，好吵。 中餐厅',
  '0,跟心灵鸡汤没什么本质区别嘛，至少我不?欢这样读经典，把经典都解读成这样有点去'],
 ['正', '的', '找', '一', '以', '9', '好', '喜'])

In [5]:
'''
step2: 数据集分割 数据迭代器构造
'''
class Dataset(Data.Dataset):
    def __init__(self,data):
        super(Dataset, self).__init__()
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x,y = self.data[index]
        # print(x,y)
        x = [s2i[i] for i in x]
        y = s2i[y]
        return torch.LongTensor(x),torch.tensor(y)

data = list(zip(origin_data,predict_word))
# 随机打乱data_list顺序
# random.shuffle(data)
# 前8000条为训练数据集
train_dataset = Dataset(data[:8425])
# 8000条后为测试数据集
test_dataset = Dataset(data[8425:])
# 训练数据集迭代器
train_loader = Data.DataLoader(
    dataset=train_dataset,
    shuffle=True,
    # 董政宇 改 2023年4月23日20:37:46
    num_workers=0,#原num_workers=0，多线程？
    batch_size=BATCH_SIZE,
    drop_last=True
)
# 测试数据集迭代器
test_loader = Data.DataLoader(
    dataset=test_dataset,
    shuffle=False,
    # 董政宇 改 2023年4月23日20:37:46
    num_workers=0,#原num_workers=0，多线程？
    batch_size=BATCH_SIZE,
    drop_last=True
)

len(test_dataset),len(train_dataset)

(33, 8425)

In [6]:
print(len(data))
# data[:10]
torch.tensor(264)
x="1,看了《饮食定生死》，觉得它是一本通俗?懂的健康科普读物。从中我认识到了食物对"
print(len(x))
x = [s2i[i] for i in x]
torch.LongTensor(x), len(torch.LongTensor(x))

8458
40


(tensor([2797, 3566, 1605, 1435, 4113, 3892,  462, 2145, 3926,  227, 2786, 2540,
         2633, 2125,  457, 1345, 3918, 3032, 2176,  950, 2040,  683,  952, 3553,
         3849, 2563,  886, 3474, 3808, 1875, 3221,   10, 2753, 3105, 1892, 1270,
         1435,  462, 3808, 3253]),
 40)

In [7]:
'''
step3: BiLSTM双向长短期记忆网络定义
'''
EPOCH = 10
import torch.nn.functional as F
# 定义BiLSTM
class BiLSTM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=WORD_DIM,
            hidden_size=NUM_HIDDENS,
            # 开启双向BiLSTM
            bidirectional=True
          )
        self.out = nn.Linear(NUM_HIDDENS*2,vocab_size)
    def forward(self,x):
        # print(x.shape)
        # x.shape [batch,seq_len,word_dim]     torch.Size([32, 40, 128])
        # state.shape [1,seq_len,word_dim]
        h_s = torch.randn((1*2,BATCH_SIZE, NUM_HIDDENS),device=device)
        c_s = torch.randn((1*2,BATCH_SIZE, NUM_HIDDENS),device=device)
        output,(_,_)= self.lstm(x.transpose(1,0),(h_s,c_s)) # output.shape [seq_len,batch,num_hiddens] torch.Size([40, 32, 256])
        chinese = self.out(output[20])  # chinese.shape [batch,vocab_size]
       
        return chinese 

# 定义encoder       
def encode(data,embed):
    return embed(data)

# 定义训练过程处理
def train_porcess(pred,acc,device):
    # pred.shape [batch,vocab_size]
    # acc.shape [batch]
    # print(pred.shape)torch.Size([32, 3195])

    pred = F.softmax(pred,dim=-1)
    # print(pred.shape)torch.Size([3195])
    pred = pred.argmax(dim=-1).cpu().numpy()
    acc = acc.cpu().numpy()
    print('预测下一个字:',[i2s[i] for i in pred.tolist()])
    print('实际下一个字:',[i2s[i] for i in acc.tolist()])

In [8]:
'''
step4: 实例化和初始化
'''
bilstm = BiLSTM(vocab_size)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(bilstm.parameters(),lr=LR)
embed = nn.Embedding(vocab_size,WORD_DIM,device = device)
output_model = 'models/BiLSTM_models/chineseBiLSTM_modelNew.pth'
COUNT = 0

In [9]:
'''
注意: 初始化模型参数（从头开始训练）
'''
torch.save(
            {
            
                'model_state_dict': bilstm.state_dict(),
                'optimizer_state_dict': optim.state_dict()
            }, output_model
          )

In [11]:
'''
step5: 开始训练(继续训练)
'''
BATCH_SIZE =32
if __name__ == '__main__':
    
    '''这段注释掉的代码意义何在 研究ing'''
    bestscore = 0
    # 加载模型（接上次训练参数继续训练？）
    checkpoint = torch.load(output_model,map_location='cpu')
    # print(checkpoint)
    bilstm.load_state_dict(checkpoint['model_state_dict'],strict=False)
    bilstm.to(device)
    bilstm.train()

    for epoch in range(10):
        # print(f'epoch = {epoch}')
        print(f'--------epoch = {COUNT*10 + epoch + 1}--------') 
        accuracy =0
        time=0
        # 训练数据集开始训练并计算loss损失  
        for step,datas in enumerate(train_loader):
            
            data,label = tuple(t.to(device)for t in datas)
            data = encode(data,embed)
            chinese = bilstm(data)
            loss = criterion(chinese,label.long())

            # # 测试
            # # print(data)
            # # print(label)
            # print("-------------")
            # print(chinese)
            # print("-------*****************------")
            # pred = F.softmax(chinese,dim=-1)
            # print(pred)
            # print(pred.argmax(dim=-1))
            # # print(pred.shape)torch.Size([3195])返回指定维度最大值的序号
            # pred = pred.argmax(dim=-1).cpu().numpy()
            # acc = label.cpu().numpy()
            # print(label)
            # print('预测下一个字:',[i2s[i] for i in pred.tolist()])
            # print('实际下一个字:',[i2s[i] for i in acc.tolist()])

            optim.zero_grad()
            loss.backward()
            optim.step()
            if step%20 == 0:
                print(f'loss = {loss}')
                
            # if step%50 == 0:
            #     train_porcess(chinese,label,device)
            # break
        
        bilstm.eval()  
        # 测试数据集开始评估并计算accuracy  
        for step,datas in enumerate(test_loader):
            # print(datas)
            data,label = tuple(t.to(device)for t in datas)
            data = encode(data,embed)
            chinese = bilstm(data)
            pred = F.softmax(chinese,dim=-1)       
            pred = pred.argmax(dim=-1).cpu().numpy()
            acc = label.cpu().numpy()
            accuracy += sum(acc==pred)
            time += len(acc)

        # 测试数据集预测
        print('预测下一个字:',[i2s[i] for i in pred.tolist()])
        print('实际下一个字:',[i2s[i] for i in acc.tolist()])
        # 测试数据集的精确度
        accuracy/=time+0.0
        print('测试数据集的accuracy为：\n',accuracy)
        #保存最优模型的参数（得分最高的模型参数）
        if accuracy>bestscore:
            torch.save(
            {
            
                'model_state_dict': bilstm.state_dict(),
                'optimizer_state_dict': optim.state_dict()
            }, output_model
          )
            bestscore = accuracy
        bilstm.train()

        # break
    COUNT = COUNT + 1

--------epoch = 11--------
loss = 3.3904199600219727
loss = 4.206971645355225
loss = 3.5998711585998535
loss = 4.4049177169799805
loss = 3.6020994186401367
loss = 4.070418834686279
loss = 3.4409632682800293
loss = 4.064728260040283
loss = 3.9831151962280273
loss = 3.5937092304229736
loss = 3.610607862472534
loss = 3.584739923477173
loss = 3.8011717796325684
loss = 3.7854278087615967
预测下一个字: ['的', '我', '是', '不', '是', '，', '的', '的', '有', '，', '一', '的', '是', '书', '。', '不', '是', '一', '不', '，', ' ', ',', '喜', '一', '间', '非', '，', ',', '是', '以', ',', '好']
实际下一个字: ['或', '了', '是', '午', '本', '见', '真', '失', '多', '级', '所', '走', ',', '不', '：', '算', '害', '线', '妹', '，', '舒', '舒', '喜', '廖', '间', '正', '的', '找', '一', '以', '9', '好']
测试数据集的accuracy为：
 0.1875
--------epoch = 12--------
loss = 3.0700008869171143
loss = 3.2430179119110107
loss = 3.6025044918060303
loss = 3.364799737930298
loss = 3.605013847351074
loss = 4.072226524353027
loss = 4.006681442260742
loss = 3.566735029220581
loss = 3.210050106048